In [7]:
import scala.util.matching
import org.apache.spark.rdd.RDD

println("all good")

all good


lastException: Throwable = null


data is in the format:

{
"conversionSymbol": "",
"volumeto": 6685400.72, 
"high": 9165.89, 
"low": 9144.6, 
"time": 1595030400, 
"volumefrom": 730.39, 
"close": 9151.68, 
"open": 9156.79, 
"conversionType": "direct"
}


In [26]:
case class TradingInfo (conversionSymbol: String, 
                        volumeto: Double, 
                        high: Double, 
                        low: Double, 
                        time: Double,
                        volumefrom: Double,
                        close: Double,
                        open: Double,
                        conversionType: String
                       )

def createTradingInfo(data: String): TradingInfo = {
    val s = data.split(", ")
    return TradingInfo(s(0).split(": ")(1), s(1).split(": ")(1).toDouble, s(2).split(": ")(1).toDouble, 
                       s(3).split(": ")(1).toDouble, s(4).split(": ")(1).toDouble, s(5).split(": ")(1).toDouble, 
                       s(6).split(": ")(1).toDouble, s(7).split(": ")(1).toDouble, s(8).split(": ")(1))
}

val filename = "../Datasets/day_json.txt"
//val johnFileName = "/home/john/ID2221-Labs/project/Datasets"
val tradingInformation = sc.textFile(filename)
            .flatMap(l => l.split("},")).
            //map(x => x.replace("\\", "")).
            map(x => x.replace("[", "")).
            map(x => x.replace("{", "")).
            map(x => x.replace("\"", "")).
            map(createTradingInfo)
            .cache()

// THE CODE IS STUPID BUT IT WORKS 
// CONVERSION SYMBOL IS EMPTY BECAUSE I REMOVED \ FROM IT

tradingInformation.take(5).foreach(println)

val test = tradingInformation.map(x => x._1)

// tradingInformation.saveAsTextFile("data_cleaned")

println("All good")

Unknown Error: <console>:55: error: value _1 is not a member of TradingInfo
       val test = tradingInformation.map(x => x._1)
                                                ^
